## Lab

_Marilyn, Shiva, Olivier_

In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
# Setup chunk

import time

# Custom utils
from utils import *

# Data wrangling
import pandas as pd
import numpy as np

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Text sanitization
import re
import nltk
from nltk.stem.snowball import SnowballStemmer

try:
    # Avoid error if you don't have the resource
    stopwords = nltk.corpus.stopwords.words("english")
except LookupError:
    nltk.download("stopwords")
    stopwords = nltk.corpus.stopwords.words("english")
    
stemmer = SnowballStemmer(language="english")

# Lang detection
#import langid
#from langid.langid import LanguageIdentifier, model
#identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)

# Misc
from tqdm import tqdm
tqdm.pandas()

# Define the seed for reproducibility
SEED = 31415

# Define n_jobs
JOBS = 7

In [39]:
# Scikit time
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)

from sklearn.feature_extraction.text import (
    CountVectorizer, 
    TfidfTransformer, 
    TfidfVectorizer
)
from sklearn.pipeline import Pipeline, make_pipeline


from sklearn.model_selection import (
    train_test_split, 
    GridSearchCV, 
    KFold, 
    cross_val_score
)

from sklearn.metrics import (
    classification_report, 
    accuracy_score, 
    confusion_matrix
)

In [40]:
df_san = pd.read_pickle("./data/sanitized.pkl")

In [41]:
df_san.head()

,Id,emotion,tweet_id,date,lyx_query,user,text,sanitized
0,0,1,2063391019,Sun Jun 07 02:28:13 PDT 2009,NO_QUERY,BerryGurus,@BreeMe more time to play with you BlackBerry ...,time play blackberri
1,1,0,2000525676,Mon Jun 01 22:18:53 PDT 2009,NO_QUERY,peterlanoie,Failed attempt at booting to a flash drive. Th...,fail attempt boot flash drive fail attempt swi...
2,2,0,2218180611,Wed Jun 17 22:01:38 PDT 2009,NO_QUERY,will_tooker,@msproductions Well ain't that the truth. Wher...,well truth damn auto lock disabl go copi past ...
3,3,1,2190269101,Tue Jun 16 02:14:47 PDT 2009,NO_QUERY,sammutimer,@Meaghery cheers Craig - that was really sweet...,cheer craig realli sweet repli pump
4,4,0,2069249490,Sun Jun 07 15:31:58 PDT 2009,NO_QUERY,ohaijustin,I was reading the tweets that got send to me w...,read tweet got send lie phone face drop amp hi...


# 2. Fitting

In [42]:
# Sample
df_sub = df_san.sample(frac=.1, random_state=SEED)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(df_sub["sanitized"], df_sub["emotion"], 
                                                    test_size=0.2, shuffle=True, random_state=SEED)

#only 4 folds because I have 4 cores, just to test
folds = KFold(n_splits=4, shuffle=True, random_state=SEED)

# Sanity check
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

X_train:  (102400,)
X_test:  (25600,)
y_train:  (102400,)
y_test:  (25600,)


## 1. Without Gridsearch

In [52]:
tfidfconverter = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', max_df=0.8, min_df=9, binary=True)
X = tfidfconverter.fit_transform(X_train)

In [53]:
clf = BernoulliNB().fit(X, y_train)

In [55]:
X_new = tfidfconverter.transform(X_test)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

ValueError: could not convert string to float: 'haha get acquaint whole '

In [29]:
clf.predict(X_new).shape
y_test.shape

(25600,)

## 2. With Gridsearch

In [ ]:
# Define the pipeline

pipe = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", BernoulliNB()),
    ]
)

start = time.time()
CV_ber = cross_val_score(
    berNB, X_train, y_train, scoring="accuracy", cv=folds, n_jobs=JOBS
)
CV_ber.fit(X_train, y_train)

